In [1]:
import os
import re
import shutil
from docx import Document
from docx.shared import Inches

# Directorio donde buscar los archivos docx
directorio_base = "C:\\Users\\Usuario\\Documents\\Github\\Camaras\\Informes"
directorio_salida = "C:\\Users\\Usuario\\Documents\\Github\\Camaras\\Informes\\Informes concatenados"

# Buscar todos los archivos docx en el directorio y subdirectorios
archivos_docx = []
for raiz, dirs, archivos in os.walk(directorio_base):
    for archivo in archivos:
        if archivo.endswith(".docx"):
            ruta_completa = os.path.join(raiz, archivo)
            print(f"Encontrado: {ruta_completa}")
            archivos_docx.append(ruta_completa)

print(f"Total de archivos encontrados: {len(archivos_docx)}")

if not archivos_docx:
    print("No se encontraron archivos DOCX en los directorios")
    exit()

# Crear un nuevo documento para el resultado
documento_final = Document()
documento_final.add_heading("Informes Concatenados", level=0)

# Para cada archivo docx encontrado
for ruta_archivo in archivos_docx:
    dir_padre = os.path.basename(os.path.dirname(ruta_archivo))
    print(f"Procesando archivo en directorio: {dir_padre}")
    
    # Verificar si coincide con el patrón "informe QQ YYYY"
    match = re.search(r"informe (\w+) (\d{4})", dir_padre, re.IGNORECASE)
    if match:
        trimestre, anno = match.groups()
        documento_final.add_heading(f"Informe del {trimestre} {anno}", level=1)
    else:
        documento_final.add_heading(f"Informe de {dir_padre}", level=1)
    
    # Añadir el nombre del archivo como subtítulo
    nombre_archivo = os.path.basename(ruta_archivo)
    documento_final.add_heading(f"Archivo: {nombre_archivo}", level=2)
    
    try:
        # Cargar el documento actual
        doc_actual = Document(ruta_archivo)
        
        # Copiar el contenido párrafo por párrafo
        for paragraph in doc_actual.paragraphs:
            # Copiar el texto y el estilo
            p = documento_final.add_paragraph()
            p.text = paragraph.text
            p.style = paragraph.style
            
            # Copiar el formato
            for run in paragraph.runs:
                r = p.add_run(run.text)
                r.bold = run.bold
                r.italic = run.italic
                r.underline = run.underline
                r.font.size = run.font.size
                r.font.name = run.font.name
                
        # Copiar tablas
        for table in doc_actual.tables:
            # Crear una nueva tabla con el mismo número de filas y columnas
            t = documento_final.add_table(rows=len(table.rows), cols=len(table.columns))
            t.style = table.style
            
            # Copiar el contenido de cada celda
            for i, row in enumerate(table.rows):
                for j, cell in enumerate(row.cells):
                    t.cell(i, j).text = cell.text
        
        # Manejar imágenes - esto es lo complicado con python-docx
        # En lugar de intentar copiar las imágenes directamente, añadimos una nota
        documento_final.add_paragraph("Nota: Las imágenes del documento original no se han incluido. Por favor, consulte el documento original para ver las imágenes.")
        
        # Añadir un salto de página después de cada documento
        documento_final.add_page_break()
        print(f"Documento añadido correctamente: {nombre_archivo}")
    except Exception as e:
        print(f"Error al procesar {ruta_archivo}: {str(e)}")
        documento_final.add_paragraph(f"Error al procesar este documento: {str(e)}")
        documento_final.add_page_break()

# Guardar el documento final
if not os.path.exists(directorio_salida):
    os.makedirs(directorio_salida)
    print(f"Creado directorio: {directorio_salida}")

ruta_documento_final = os.path.join(directorio_salida, "Informes_Concatenados.docx")
documento_final.save(ruta_documento_final)
print(f"Guardado documento final en: {ruta_documento_final}")

# Crear un HTML simple
ruta_html = os.path.join(directorio_salida, "Informes_Concatenados.html")
with open(ruta_html, "w", encoding="utf-8") as archivo_html:
    archivo_html.write("<html><body>\n")
    archivo_html.write("<h1>Informes Concatenados</h1>\n")
    archivo_html.write("<p>Se han concatenado los siguientes informes:</p>\n")
    archivo_html.write("<ul>\n")
    for ruta in archivos_docx:
        nombre_archivo = os.path.basename(ruta)
        dir_padre = os.path.basename(os.path.dirname(ruta))
        archivo_html.write(f"<li>{nombre_archivo} (de {dir_padre})</li>\n")
    archivo_html.write("</ul>\n")
    archivo_html.write("<p><strong>Nota:</strong> Las imágenes no se han incluido en el documento concatenado.</p>\n")
    archivo_html.write("</body></html>\n")
print(f"Creado archivo HTML en: {ruta_html}")

print("Proceso completado.")

Encontrado: C:\Users\Usuario\Documents\Github\Camaras\Informes\agregado\agregado.docx
Encontrado: C:\Users\Usuario\Documents\Github\Camaras\Informes\Informe 1q 2022\informe iq 2022 v4.docx
Encontrado: C:\Users\Usuario\Documents\Github\Camaras\Informes\Informe 1q 2024\Informe IQ 24.docx
Encontrado: C:\Users\Usuario\Documents\Github\Camaras\Informes\Informe 2q 2022\informe iiq 2022 v4. Revisado Curro.docx
Encontrado: C:\Users\Usuario\Documents\Github\Camaras\Informes\Informe 2q 2023\informe 2q 2023.docx
Encontrado: C:\Users\Usuario\Documents\Github\Camaras\Informes\Informe 2q 2024\Informe 2Q 24.docx
Encontrado: C:\Users\Usuario\Documents\Github\Camaras\Informes\Informe 3q 2022\informe 3q 2022 v4.docx
Encontrado: C:\Users\Usuario\Documents\Github\Camaras\Informes\Informe 3q 2023\informe 3q 2023 v5.docx
Encontrado: C:\Users\Usuario\Documents\Github\Camaras\Informes\Informe 4q 2022\informe 4q 2022 v5.docx
Encontrado: C:\Users\Usuario\Documents\Github\Camaras\Informes\Informe 4q 2023\Informe

In [5]:
import mammoth
import os

# Define las rutas de tu archivo de entrada (.docx) y salida (.html)
# Asegúrate de cambiar las rutas según tu caso
input_filename = r"C:\Users\Usuario\Documents\Github\Camaras\Informes\Informes concatenados\Informes_Concatenados.docx"
output_filename = r"C:\Users\Usuario\Documents\Github\Camaras\Informes\Informes concatenados\Informes_Concatenados_mammoth.html" # Usamos un nombre diferente para no sobrescribir

# Verifica si el archivo de entrada existe
if not os.path.exists(input_filename):
    print(f"Error: El archivo de entrada no se encuentra en la ruta especificada: {input_filename}")
else:
    try:
        print(f"Intentando convertir '{input_filename}' a HTML usando mammoth...")

        # Realiza la conversión
        # mammoth.convert devuelve un objeto con el HTML en .value y mensajes en .messages
        with open(input_filename, "rb") as docx_file: # Abrir en modo binario 'rb'
             result = mammoth.convert(docx_file)
             html_content = result.value # El contenido HTML generado
             messages = result.messages # Mensajes de advertencia o error durante la conversión

        # Guarda el contenido HTML en un archivo de salida
        with open(output_filename, "w", encoding="utf-8") as html_file:
            html_file.write(html_content)

        print(f"¡Conversión exitosa con mammoth! Archivo HTML guardado como: {output_filename}")
        if messages:
            print("\nMensajes de la conversión (pueden ser advertencias):")
            for message in messages:
                print(f"- {message}")

    except Exception as e:
        print(f"Ocurrió un error durante la conversión con mammoth: {e}")

Intentando convertir 'C:\Users\Usuario\Documents\Github\Camaras\Informes\Informes concatenados\Informes_Concatenados.docx' a HTML usando mammoth...
¡Conversión exitosa con mammoth! Archivo HTML guardado como: C:\Users\Usuario\Documents\Github\Camaras\Informes\Informes concatenados\Informes_Concatenados_mammoth.html

Mensajes de la conversión (pueden ser advertencias):
- Message(type='warning', message='Paragraph style with ID BodyA was referenced but not defined in the document')
- Message(type='warning', message='Paragraph style with ID s2 was referenced but not defined in the document')
- Message(type='warning', message='Table style with ID Tablanormal was referenced but not defined in the document')
- Message(type='warning', message='Unrecognised paragraph style: Title (Style ID: Title)')
- Message(type='warning', message='Unrecognised paragraph style: None (Style ID: BodyA)')
- Message(type='warning', message='Unrecognised paragraph style: None (Style ID: s2)')
